In [1]:
import os
import pandas as pd
import glob
import math
import numpy as np

In [2]:
try:
    metadata_2011 = pd.read_csv(r'.\data\raw\api\msoa\counts\c2011\metadata.csv')
except:  
    os.chdir("..")
    metadata_2011 = pd.read_csv(r'.\data\raw\api\msoa\counts\c2011\metadata.csv')
metadata_2011['clean_mnemonic'] = metadata_2011['mnemonic'].str.extract('(c2011ks\d*).*')
metadata_2011 = metadata_2011[metadata_2011['geoglevel'].str.contains('oa')]
metadata_2011_mnemonics = list(set(metadata_2011['clean_mnemonic']))
metadata_2011['geoglevel']

0     oa,ps
1     oa,ps
2     oa,ps
3     oa,ps
4     oa,ps
5     oa,ps
6     oa,ps
7     oa,ps
8     oa,ps
9     oa,ps
10    oa,ps
11    oa,ps
12    oa,ps
13    oa,ps
14    oa,ps
15    oa,ps
16    oa,ps
17    oa,ps
18    oa,ps
19    oa,ps
20    oa,ps
21    oa,ps
22    oa,ps
23    oa,ps
24    oa,ps
25    oa,ps
26    oa,ps
27    oa,ps
28    oa,ps
29       oa
30       oa
31       oa
32       oa
33       oa
34       oa
35       oa
36       oa
37       oa
38       oa
39       oa
40       oa
41       oa
42       oa
43       oa
44       oa
45       oa
46       oa
47       oa
48       oa
49       oa
50       oa
51       oa
52       oa
Name: geoglevel, dtype: object

In [3]:
c2011_dfs = []

for mnemonic in metadata_2011_mnemonics:

    root_dir = r'.\data\raw\api\msoa\counts\c2011'
    mnemonic_tables = glob.glob(mnemonic+'*', root_dir=root_dir)
    mnemonic_dfs = []

    for table in mnemonic_tables:
        df = pd.read_csv(rf"{root_dir}\{table}")
        mnemonic_dfs.append(df)

    print(mnemonic)
    mnemonic_df = pd.concat(mnemonic_dfs)

    mnemonic_df = mnemonic_df.drop(columns=['Unnamed: 0', 'GEOGRAPHY_TYPE', 'GEOGRAPHY_TYPECODE', 'DATE'])
    mnemonic_df['OBS_VALUE'] = mnemonic_df['OBS_VALUE'].astype(float)
    mnemonic_df = mnemonic_df[mnemonic_df['OBS_VALUE'] > 0]
    cols = list(mnemonic_df.columns)
    var_col = [col for col in cols if col not in ['GEOGRAPHY_CODE', 'OBS_VALUE']][0]
    mnemonic_df = mnemonic_df.pivot_table(index='GEOGRAPHY_CODE', columns=var_col, values='OBS_VALUE', aggfunc='mean', fill_value=0)
    mnemonic_df = mnemonic_df.reset_index(drop=False)
    c2011_dfs.append(mnemonic_df)
c2011_dfs[0]

c2011ks104
c2011ks208
c2011ks205
c2011ks602
c2011ks606
c2011ks207
c2011ks501
c2011ks403
c2011ks102
c2011ks209
c2011ks105
c2011ks607
c2011ks404
c2011ks613
c2011ks605
c2011ks106
c2011ks204
c2011ks609
c2011ks610
c2011ks107
c2011ks601
c2011ks608
c2011ks103
c2011ks612
c2011ks101
c2011ks603
c2011ks402
c2011ks401
c2011ks301
c2011ks202
c2011ks405
c2011ks206
c2011ks611
c2011ks604
c2011ks201


CELL_NAME,GEOGRAPHY_CODE,All categories: Living arrangements,Living in a couple,Living in a couple: Cohabiting,Living in a couple: Married or in a registered same-sex civil partnership,Not living in a couple,Not living in a couple: Divorced or formerly in a same-sex civil partnership which is now legally dissolved,Not living in a couple: Married or in a registered same-sex civil partnership,Not living in a couple: Separated (but still legally married or still legally in a same-sex civil partnership),Not living in a couple: Single (never married or never registered a same-sex civil partnership),Not living in a couple: Widowed or surviving partner from a same-sex civil partnership
0,E02000001,6595.0,2848.0,894.0,1954.0,3747.0,430.0,371.0,141.0,2537.0,268.0
1,E02000002,4973.0,2350.0,508.0,1842.0,2623.0,430.0,98.0,156.0,1527.0,412.0
2,E02000003,7786.0,4032.0,670.0,3362.0,3754.0,535.0,218.0,202.0,2292.0,507.0
3,E02000004,4741.0,2632.0,458.0,2174.0,2109.0,283.0,78.0,100.0,1312.0,336.0
4,E02000005,6362.0,3270.0,678.0,2592.0,3092.0,421.0,145.0,203.0,1912.0,411.0
...,...,...,...,...,...,...,...,...,...,...,...
7196,W02000419,8748.0,5301.0,1061.0,4240.0,3447.0,658.0,79.0,166.0,1838.0,706.0
7197,W02000420,10898.0,7370.0,1012.0,6358.0,3528.0,628.0,94.0,141.0,1975.0,690.0
7198,W02000421,9531.0,6026.0,1004.0,5022.0,3505.0,588.0,135.0,138.0,1907.0,737.0
7199,W02000422,5579.0,2572.0,1570.0,1002.0,3007.0,307.0,154.0,145.0,2356.0,45.0


In [4]:
for df in c2011_dfs:
    print(df.columns)
    total_col = df.filter(regex='(^All categories.*)|(.*All households.*)|(All usual residents.*)|(All lone parent housholds.*)').columns[0]
    total_cols = list(df.filter(regex='(^All categories.*)|(.*All households.*)|(All usual residents.*)|(All lone parent housholds.*)').columns)
    agg_cols = list(df.filter(regex='.*([Aa]verage)|([Mm]ean)|([Mm]edian)|([Nn]umber of).*').columns)
    geo_col = 'GEOGRAPHY_CODE'
    val_cols = [col for col in df.columns if col not in total_cols and col not in agg_cols and col != geo_col]
    for col in val_cols:
        df[col] = df[col] / df[total_col]

Index(['GEOGRAPHY_CODE', 'All categories: Living arrangements',
       'Living in a couple', 'Living in a couple: Cohabiting',
       'Living in a couple: Married or in a registered same-sex civil partnership',
       'Not living in a couple',
       'Not living in a couple: Divorced or formerly in a same-sex civil partnership which is now legally dissolved',
       'Not living in a couple: Married or in a registered same-sex civil partnership',
       'Not living in a couple: Separated (but still legally married or still legally in a same-sex civil partnership)',
       'Not living in a couple: Single (never married or never registered a same-sex civil partnership)',
       'Not living in a couple: Widowed or surviving partner from a same-sex civil partnership'],
      dtype='object', name='CELL_NAME')
Index(['GEOGRAPHY_CODE', 'Age 16 to 64', 'Age 3 to 15', 'Age 65 and over',
       'All categories: Age 3 and over',
       'All categories: Can speak Welsh by age 3 and over',
       'A

In [5]:
for mnemonic, df in zip(metadata_2011_mnemonics, c2011_dfs):
    df.to_csv(rf'.\data\raw\api\msoa\counts\c2011\{mnemonic}.csv')

In [6]:
try:
    metadata_2001 = pd.read_csv(r'.\data\raw\api\msoa\counts\c2001\metadata.csv')
except:  
    os.chdir("..")
    metadata_2001 = pd.read_csv(r'.\data\raw\api\msoa\counts\c2001\metadata.csv')
metadata_2001['clean_mnemonic'] = metadata_2001['mnemonic'].str.extract('(ks\d*).*')
metadata_2001 = metadata_2001[metadata_2001['geoglevel'].str.contains('msoa')]
metadata_2001_mnemonics = list(set(metadata_2001['clean_mnemonic']))
len(metadata_2001_mnemonics)

24

In [7]:
c2001_dfs = []

for mnemonic in metadata_2001_mnemonics:
    root_dir = r'.\data\raw\api\msoa\counts\c2001'
    mnemonic_tables = glob.glob(mnemonic+'*', root_dir=root_dir)
    mnemonic_dfs = []

    for table in mnemonic_tables:
        df = pd.read_csv(rf"{root_dir}\{table}")
        mnemonic_dfs.append(df)

    mnemonic_df = pd.concat(mnemonic_dfs)

    mnemonic_df = mnemonic_df.drop(columns=['Unnamed: 0', 'GEOGRAPHY_TYPE', 'GEOGRAPHY_TYPECODE', 'DATE'])
    mnemonic_df['OBS_VALUE'] = mnemonic_df['OBS_VALUE'].astype(float)
    mnemonic_df = mnemonic_df[mnemonic_df['OBS_VALUE'] > 0]
    cols = list(mnemonic_df.columns)
    var_col = [col for col in cols if col not in ['GEOGRAPHY_CODE', 'OBS_VALUE']][0]
    mnemonic_df = mnemonic_df.pivot_table(index='GEOGRAPHY_CODE', columns=var_col, values='OBS_VALUE', aggfunc='mean', fill_value=0)
    mnemonic_df = mnemonic_df.reset_index(drop=False)
    c2001_dfs.append(mnemonic_df)
c2001_dfs[0]

CELL_NAME,GEOGRAPHY_CODE,All usual residents aged 16 to 74,Economically Inactive,Economically active,Economically active: Employee: Full-time,Economically active: Employee: Part-time,Economically active: Full-time student,Economically active: In employment,Economically active: Self-employed,Economically active: Unemployed,Economically inactive: Long-term sick or disabled,Economically inactive: Looking after home or family,Economically inactive: Other,Economically inactive: Retired,Economically inactive: Student (including full-time students),Long-term unemployed,Unemployed: Age 16 to 24,Unemployed: Age 50 to 74,Unemployed: Never worked
0,E02000001,4044.666667,1021.333333,3023.333333,2026.000000,184.000000,78.000000,2801.333333,591.333333,144.000000,120.000000,154.666667,140.000000,364.000000,242.666667,33.333333,28.000000,31.333333,19.000000
1,E02000002,2748.000000,1151.333333,1596.666667,956.000000,278.000000,52.000000,1404.666667,170.666667,140.000000,207.333333,282.666667,116.000000,452.666667,92.666667,50.666667,38.666667,20.000000,15.333333
2,E02000003,4395.333333,1402.000000,2993.333333,1943.333333,463.333333,111.333333,2753.333333,346.666667,128.666667,190.000000,341.333333,152.000000,558.666667,160.000000,36.666667,46.000000,15.333333,9.333333
3,E02000004,2873.333333,911.333333,1962.000000,1308.000000,322.000000,53.333333,1827.333333,197.333333,81.333333,160.000000,194.000000,79.333333,393.333333,84.666667,23.333333,27.333333,8.666667,5.666667
4,E02000005,3657.333333,1319.333333,2338.000000,1482.666667,412.666667,75.333333,2142.666667,247.333333,120.000000,251.333333,328.666667,124.666667,484.666667,130.000000,35.333333,38.000000,16.666667,14.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7189,W02000409,2736.666667,1066.000000,1670.666667,1008.000000,372.000000,54.666667,1536.000000,156.000000,80.000000,217.333333,229.333333,126.000000,397.333333,96.000000,18.666667,26.666667,12.333333,8.666667
7190,W02000410,3238.666667,1310.666667,1928.000000,1140.666667,332.000000,109.333333,1676.000000,203.333333,142.666667,314.000000,299.333333,165.333333,303.333333,228.666667,47.333333,34.666667,18.666667,15.333333
7191,W02000411,2665.333333,1062.000000,1603.333333,954.666667,345.333333,55.333333,1424.666667,124.666667,123.333333,299.333333,238.666667,156.000000,272.000000,96.000000,44.000000,30.666667,20.666667,14.666667
7192,W02000412,2755.333333,1008.666667,1746.666667,1085.333333,302.666667,74.666667,1562.666667,174.666667,109.333333,262.666667,210.666667,121.333333,308.666667,105.333333,32.000000,24.000000,11.333333,11.333333


In [8]:
for df in c2001_dfs:
    print(df.columns)
    total_col = df.filter(regex='(^All categories.*)|(.*with residents.*)|(All usual residents.*)|(Number of residents by communal establishment type.*)|(All people.*)|(All households.*)|(All males.*)|(All lone parent housholds.*)').columns[0]
    total_cols = list(df.filter(regex='(^All categories.*)|(.*with residents.*)|(All usual residents.*)|(Number of residents by communal establishment type.*)|(All people.*)|(All households.*)|(All males.*)|(All lone parent housholds.*)').columns)
    print(total_col)
    agg_cols = list(df.filter(regex='.*([Aa]verage)|([Mm]ean)|([Mm]edian)|([Nn]umber of).*').columns)
    geo_col = 'GEOGRAPHY_CODE'
    val_cols = [col for col in df.columns if col not in total_cols and col not in agg_cols and col != geo_col]
    for col in val_cols:
        df[col] = df[col] / df[total_col]

Index(['GEOGRAPHY_CODE', 'All usual residents aged 16 to 74',
       'Economically Inactive', 'Economically active',
       'Economically active: Employee: Full-time',
       'Economically active: Employee: Part-time',
       'Economically active: Full-time student',
       'Economically active: In employment',
       'Economically active: Self-employed', 'Economically active: Unemployed',
       'Economically inactive: Long-term sick or disabled',
       'Economically inactive: Looking after home or family',
       'Economically inactive: Other', 'Economically inactive: Retired',
       'Economically inactive: Student (including full-time students)',
       'Long-term unemployed', 'Unemployed: Age 16 to 24',
       'Unemployed: Age 50 to 74', 'Unemployed: Never worked'],
      dtype='object', name='CELL_NAME')
All usual residents aged 16 to 74
Index(['GEOGRAPHY_CODE', 'All households', 'Average household size',
       'Average number of rooms per household',
       'Lowest floor level

In [9]:
for mnemonic, df in zip(metadata_2001_mnemonics, c2001_dfs):
    df.to_csv(rf'.\data\raw\api\msoa\percents\c2001\{mnemonic}.csv')

In [10]:
try:
    metadata_2021 = pd.read_csv(r'.\data\raw\api\msoa\counts\c2021\metadata.csv')
except:  
    os.chdir("..")
    metadata_2021 = pd.read_csv(r'.\data\raw\api\msoa\counts\c2021\metadata.csv')
metadata_2021['clean_mnemonic'] = metadata_2021['mnemonic'].str.extract('(c2021ts.*)')
metadata_2021 = metadata_2021[metadata_2021['geoglevel'].str.contains('msoa')]
metadata_2021_mnemonics = list(set(metadata_2021['clean_mnemonic']))
len(metadata_2021_mnemonics)

68

In [11]:
c2021_dfs = []

for mnemonic in metadata_2021_mnemonics:
    root_dir = r'.\data\raw\api\msoa\counts\c2021'
    mnemonic_tables = glob.glob(mnemonic+'*', root_dir=root_dir)
    mnemonic_dfs = []

    for table in mnemonic_tables:
        df = pd.read_csv(rf"{root_dir}\{table}")
        mnemonic_dfs.append(df)

    mnemonic_df = pd.concat(mnemonic_dfs)

    mnemonic_df = mnemonic_df.drop(columns=['Unnamed: 0', 'GEOGRAPHY_TYPE', 'GEOGRAPHY_TYPECODE', 'DATE'])
    mnemonic_df['OBS_VALUE'] = mnemonic_df['OBS_VALUE'].astype(float)
    mnemonic_df = mnemonic_df[mnemonic_df['OBS_VALUE'] > 0]
    cols = list(mnemonic_df.columns)
    var_col = [col for col in cols if col not in ['GEOGRAPHY_CODE', 'OBS_VALUE']][0]
    mnemonic_df = mnemonic_df.pivot_table(index='GEOGRAPHY_CODE', columns=var_col, values='OBS_VALUE', aggfunc='mean', fill_value=0)
    mnemonic_df = mnemonic_df.reset_index(drop=False)
    c2021_dfs.append(mnemonic_df)
c2021_dfs[37]

C2021_RELIGION_10_NAME,GEOGRAPHY_CODE,Buddhist,Christian,Hindu,Jewish,Muslim,No religion,Not answered,Other religion,Sikh,Total: All usual residents
0,E02000001,95.0,2976.0,203.0,177.0,540.0,3763.0,767.0,55.0,6.0,8582.0
1,E02000002,27.0,3899.0,269.0,30.0,1610.0,1816.0,426.0,32.0,172.0,8281.0
2,E02000003,35.0,4961.0,571.0,18.0,3072.0,1576.0,589.0,56.0,665.0,11543.0
3,E02000004,33.0,3456.0,163.0,13.0,829.0,1671.0,353.0,18.0,104.0,6640.0
4,E02000005,46.0,4587.0,744.0,22.0,2387.0,2210.0,605.0,78.0,405.0,11084.0
...,...,...,...,...,...,...,...,...,...,...,...
7259,W02000424,19.0,3923.0,23.0,2.0,32.0,2226.0,434.0,20.0,4.0,6683.0
7260,W02000425,34.0,4394.0,22.0,10.0,195.0,3164.0,463.0,45.0,18.0,8345.0
7261,W02000426,23.0,5416.0,21.0,2.0,93.0,3648.0,665.0,26.0,2.0,9896.0
7262,W02000427,25.0,4311.0,11.0,3.0,23.0,6015.0,679.0,69.0,7.0,11143.0


In [12]:
for df in c2021_dfs:
    geo_col = 'GEOGRAPHY_CODE'
    try:
        total_col = df.filter(regex='Total.*').columns[0]
    except:
        total_col = [col for col in df.columns if col not in [geo_col]][0]
    total_cols = list(df.filter(regex='Total.*').columns)
    agg_cols = list(df.filter(regex='.*([Aa]verage)|([Mm]ean)|([Mm]edian)|([Nn]umber of)|(\d\sbedrooms?)|(more\sbedrooms).*').columns)
    geo_col = 'GEOGRAPHY_CODE'
    val_cols = [col for col in df.columns if col not in total_cols and col not in agg_cols and col != geo_col and col != total_col]
    for col in val_cols:
        df[col] = df[col] / df[total_col]

In [13]:
for mnemonic, df in zip(metadata_2021_mnemonics, c2021_dfs):
    print(mnemonic)
    print(df.head())
    df.to_csv(rf'.\data\raw\api\msoa\percents\c2021\{mnemonic}.csv')

c2021ts034
C2021_WLANGW_3_NAME GEOGRAPHY_CODE  Can write Welsh  Cannot write Welsh  \
0                        W02000001         0.492298            0.507702   
1                        W02000002         0.461724            0.538276   
2                        W02000003         0.304090            0.695910   
3                        W02000004         0.404056            0.595944   
4                        W02000005         0.393410            0.606590   

C2021_WLANGW_3_NAME  Total: All usual residents aged 3 years and over  
0                                                              8180.0  
1                                                              6231.0  
2                                                             10270.0  
3                                                              7249.0  
4                                                              8012.0  
c2021ts066
C2021_EASTAT_20_NAME GEOGRAPHY_CODE  \
0                         E02000001   
1                